In [1]:
import datetime as dt # date and time tools

from collections import namedtuple

from os import path

import numpy as np
from scipy.integrate import solve_ivp
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import io, base64
from IPython.display import Image, HTML

# alguns textos
aa

In [2]:
#beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD

In [3]:
num_pop = 300
beta = 0.5
theta = 0.2
p = 0.1
lamb = 0.2
sigma = 0.2
rho = 0.5
epsA = 0.1
gammaA = 0.4
epsI = 0.3
gammaI = 0.2
dI = 0.1
gammaD = 0.2
dD = 0.2
I_0 = 8
T = 80
tempos = np.array(range(T+1))
num_sim = 60

In [4]:
pop_0 = np.ones(num_pop)
infectados_0 = np.random.choice(num_pop, I_0)
pop_0[infectados_0] = 3*np.ones(I_0)
attr_pop_0 = dict([(i, {'estado': int(pop_0[i])}) for i in range(num_pop)])
print(pop_0)

[1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1.
 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
def evolucao_SEIRQAD(pop_0, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD, tempos, plot=False):

    SIR_Compartimental = namedtuple('SEIRQAD_Compartimental', 
                                 [
                                     'pop_0',
                                     'num_pop',
                                     'beta',
                                     'theta',
                                     'p',
                                     'lamb',
                                     'sigma',
                                     'rho',
                                     'epsA',
                                     'gammaA',
                                     'epsI',
                                     'gammaI',
                                     'dI',
                                     'gammaD',
                                     'dD',
                                     't_instantes',
                                     'S',
                                     'E'
                                     'I', 
                                     'R',
                                     'Q',
                                     'A',
                                     'D',
                                     'X'
                                 ])        
          
    def F(t, v, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, deathI, gammaD, deathD):
        """Differential equation for SEIR-QAD model"""
        S,Q,E,A,I,D,R = v
        dS = - beta*S*(I + theta*A) - p*S + lamb*Q
        dQ = p*S - lamb*Q
        dE = beta*S*(I + theta*A) - sigma*E
        dA = sigma*(1 - rho)*E - epsA*A - gammaA*A
        dI = sigma*rho*E - epsI*I - gammaI*I - deathI*I
        dD = epsA*A + epsI*I - gammaD*D - deathD*D
        dR = gammaA*A + gammaI*I + gammaD*D
        return np.array([dS, dQ, dE, dA, dI, dD, dR])   
    
    y0 = pop_0[0:7]
    sol = solve_ivp(diferencial, t_span=[tempos[0],tempos[-1]], 
                    y0 = y0, t_eval = tempos,
                    args=(beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, deathI, gammaD, deathD))
    
    resultado = SEIRQAD_Compartimental(pop_0, num_pop, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI,
                                       deathI, gammaD, deathD, tempos,
                                       sol.y[0], sol.y[1], sol.y[2], sol.y[3], sol.y[4], sol.y[5], sol.y[6], sol)
    
    return resultado

In [6]:
def analise_grafo(G, info=True, node_size=0, pos=None, hist=False):

    num_vertices = G.number_of_nodes()
    num_arestas = G.number_of_edges()
    num_arestas_por_vertice = [j for i,j in G.degree]
    num_medio_conexoes = 2*num_arestas/num_vertices

    if info:
        print(f'Número de arestas: {num_arestas}')
        print(f'Número de vértices: {num_vertices}')
        print(f'Número médio de conexões por indivíduo: {num_medio_conexoes:.1f}')

    if node_size:
        color = ['tab:blue', 'tab:red', 'tab:green']
        pop_estado = nx.get_node_attributes(G,'estado')
        color_map = [color[pop_estado[j]-1] for j in range(num_vertices)]
        plt.figure(figsize=(10,6))
        if pos:
            nx.draw(G, pos, node_size=node_size, node_color=color_map, alpha=0.5)
        else:
            nx.draw(G, node_size=node_size, node_color=color_map, alpha=0.5)
        plt.title('Rede de indivíduos e de suas conexões', fontsize=16)
        plt.show()

    if hist:
        plt.figure(figsize=(10,6))
        plt.hist(num_arestas_por_vertice, 50, facecolor='g', alpha=0.75)
        plt.xlabel('num. arestas', fontsize=14)
        plt.ylabel('num. vertices', fontsize=14)
        plt.title('Histograma do número vertices por número de arestas', fontsize=16)
        plt.show()
    return num_medio_conexoes

In [ ]:
def evolucao_grafo_estruturado(pop_0, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, G, tempos, num_sim, show=''):
    """Evolução temporal da epidemia em um grafo estruturado.


    Entrada:
        pop_0: numpy.array
            state of the population, with 
                1: suscetível
                2: infectado
                3: recuperado ou removido

        mu: float
            probabilidade de contágio por contato

        gamma: float
            taxa de recuperação por unidade de tempo

        G: numpy.array
            grafo de conexões

        tempos: numpy.array
            instantes de tempo

        num_sim: int
            número de simulações

        show: str
            indica se é para exibir um gráfico e de que tipo:
                - 'nuvem': exibe uma nuvem com todas as simulações e o valor médio em destaque
                - 'sd': exibe o valor médio com um intervalo de confiança dado pelo desvio padrão
                - 'medio': exibe apenas o valor médio
                - '': não exibe gráfico algum.

    Saída
        X: class.SIR_Individual
            Uma instância da classe `SIR_Individual` com os seguintes atributos:
                pop_0:
                num_pop:
                mu:
                gamma:
                G:
                tempos:
                num_sim:
                S_mean:
                I_mean: 
                R_mean:
                I_sigma:
                R_sigma:
                S_sigma:
    """

    def passo_grafo_estruturado(num_pop, populacao, A_adj, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD):
        """
        DEFINITIVAMENTE NÃO ESTÁ OTIMIZADO PARA VELOCIDADE DE PROCESSAMENTO E MEMÓRIA
        
        Foi otimizado para facilidade de construção.
        
        Por exemplo, geramos muito mais números aleatórios do que realmente precisamos. 
        Precisamos de, no máximo, num_edges + num_pop número aleatórios, mas geramos num_pop**2.
        """
        I = np.eye(num_pop)
        # gera uma matriz cheia aleatória (números em [0.0,1.0))
        A_random = np.random.rand(num_pop, num_pop)
        
        # filtra de acordo com as conexões
        A_graph_random = np.multiply(A_adj, A_random)
        
        # copia a população (acho que é desnecessário)
        pop_aux = np.copy(populacao)
        
        # separa os suscetíveis, criando um vetor de 1's e 0's, se for, ou não, suscetível
        pop_suscetiveis = np.select([pop_aux==1], [pop_aux])
        
        # separa os infectados, criando um vetor de 1's e 0's, se for, ou não, infectado
        pop_infectados = np.select([pop_aux==3], [pop_aux])/3
        
        # separa os assintomaticos, criando um vetor de 1's e 0's, se for, ou não, infectado assintomático
        pop_assintomaticos = np.select([pop_aux==4], [pop_aux])/4
        
        # cria uma matriz de risco, mantendo apenas as conexões que envolvem um infectado
        A_risco = np.multiply(np.tile(pop_infectados, (num_pop, 1)), A_graph_random)
        
        # cria uma matriz de risco, mantendo apenas as conexões que envolvem um infectado assintomatico
        B_risco = np.multiply(np.tile(pop_assintomaticos, (num_pop, 1)), A_graph_random)
        
        # cria matriz de contatos entre um suscetível e um infectado
        A_contatos = np.multiply(np.tile(pop_suscetiveis, (num_pop, 1)).transpose(), A_risco)
        
        # cria matriz de contatos entre um suscetível e um assintomatico
        B_contatos = np.multiply(np.tile(pop_suscetiveis, (num_pop, 1)).transpose(), B_risco)
        
        # cria uma matriz de 1's e 0's, indicando se houve, ou não, contágio por um infectado
        A_infectados = np.select([A_contatos > 1-prob_infeccao], [np.ones([num_pop, num_pop])])
        
        # cria uma matriz de 1's e 0's, indicando se houve, ou não, contágio por um assintomatico
        B_infectados = np.select([B_contatos > 1-prob_infeccao_assint], [np.ones([num_pop, num_pop])])

        # obtém novos expostos
        pop_novos_expostos = np.select([np.sum(A_infectados + B_infectados, axis=1)>0], [np.ones(num_pop)])
        
        # separa os epostos, criando um vetor de 1's e 0's, se for, ou não, exposto
        pop_expostos = np.select([pop_aux==2], [pop_aux])/2
        
        # separa os diagnosticados, criando um vetor de 1's e 0's, se for, ou não, diagnosticado
        pop_diagnosticados = np.select([pop_aux==5], [pop_aux])/5
        
        # filtra matriz aleatória com a diagonal
        pop_recuperando_inf = pop_infectados @ np.multiply(I, A_random)
        pop_recuperando_assint = pop_assintomaticos @ np.multiply(I, A_random)
        pop_recuperando_diagnost = pop_diagnosticados @ np.multiply(I, A_random)
        pop_incubacao = pop_expostos @ np.multiply(I, A_random)
 
        
        # obtém novos recuperados
        pop_novos_recuperados_inf = np.select([pop_recuperando_inf > 1-prob_recuperacao_inf], [np.ones(num_pop)])
        pop_novos_recuperados_assint = np.select([pop_recuperando_assint > 1-prob_recuperacao_assint], [np.ones(num_pop)])
        pop_novos_recuperados_diagnost = np.select([pop_recuperando_diagnost > 1-prob_recuperacao_diagnost], [np.ones(num_pop)])
        
        pop_novos_recuperados = pop_novos_recuperados_inf + pop_novos_recuperados_assint + pop_novos_recuperados_diagnost
        
        # obtém novos infectados e assintomáticos
        pop_novos_infs = np.multiply(np.select([pop_incubacao > 1-prob_incubacao], [np.ones(num_pop)]), np.random.rand(num_pop))
        pop_novos_infectados = np.select([pop_novos_infs < proporcao_inf], np.ones(num_pop))
        pop_novos_assintomaticos = np.select([pop_novos infs >= proporcao_inf], [np.ones(num_pop)])
        
        # separa os suscetíveis, criando um vetor de 1's e 0's, se for, ou não, exposto
        pop_suscetiveis = np.select([pop_aux==1], [pop_aux])
        
        # separa os quarentenados, criando um vetor de 1's e 0's, se for, ou não, exposto
        pop_quarentenados = 1 + np.select([pop_aux==0], [pop_aux])
        
        # filtra matriz aleatória com a diagonal
        pop_quarentenando = pop_suscetiveis @ np.multiply(I, A_random)
        pop_saindo_quar = pop_quarentenados @ np.multiply(I, A_random)

        # obtém novos quarentenados
        pop_novos_quarentenados = np.select([np.logical_and(pop_quarentenando > 1-prob_entrar_quar, pop_novos_expostos == 0)], [np.ones(num_pop)])
        
        # obtém novos suscetíveis
        pop_novos_suscetiveis = np.select([pop_saindo_quar <1 - prob_sair_quar], [np.ones(num_pop)])
        
        # obtém novos diagnosticados
        diagnost_random = np.random.rand(num_pop)
        pop_diagnosticando_inf = np.multiply(pop_infectados, diagnost_random)
        pop_diagnosticando_assint = np.multiply(pop_assint, diagnost_random)
        pop_novos_diagnost_inf = np.select([np.logical_and(pop_diagnost_inf > 1-prob_diagnost_inf, pop_novos_recuperados == 0)], [np.ones(num_pop)])
        pop_novos_diagnost_assint = np.select([np.logical_and(pop_diagnost_assint > 1-prob_diagnost_assint, pop_novos_recuperados == 0)], [np.ones(num_pop)])
        pop_novos_diagnosticados = pop_novos_diagnost_inf + pop_novos_diagnost_assint
        
        # obtém novos obitos
        obitos_random = np.random.rand(num_pop)
        pop_obit_inf = np.multiply(pop_infectados, obitos_random)
        pop_obit_diagnost = np.multiply(pop_diagnosticados, obitos_random)
        pop_novos_obitos_inf = np.select([np.logical_and(pop_obit_inf > 1-prob_obit_inf, pop_novos_recuperados == 0, pop_novos_diagnosticados == 0)], [np.ones(num_pop)])
        pop_novos_obitos_diagnost = np.select([np.logical_and(pop_obit_diagnost > 1-prob_obit_diagnost, pop_novos_recuperados == 0)], [np.ones(num_pop)])
        pop_novos_obitos = pop_novos_obitos_inf + pop_novos_obitos_diagnost
        
        
        # atualiza população adicionando um aos que avançaram de estágio
        populacao_nova = pop_novos_suscetiveis + 2*pop_novos_expostos + 3*pop_novos_infectados + 4*pop_novos_assintomaticos + 5*pop_novos_diagnosticados + 
        

        # Observe que cada elemento da matriz aleatória é usado apenas uma vez, garantindo
        # a independência desses eventos aleatórios (tanto quanto se leve em consideração
        # que os números gerados são pseudo-aleatórios)

        return populacao_nova

    
    # confere se escolha para `show` é válida
    if show:
        assert(show in ('nuvem', 'sd', 'media')), 'Valor inválido para argumento `show`.'

    # atributos de saída
    SIR_Individual = namedtuple('SIR_Individual', 
                                [
                                    'pop_0',
                                    'num_pop',
                                    'mu',
                                    'gamma',
                                    'G',
                                    'tempos',
                                    'num_sim',
                                    'S_mean',
                                    'I_mean', 
                                    'R_mean',
                                    'S_sigma',
                                    'I_sigma', 
                                    'R_sigma'
                                ])
    
    # número de indivíduos da população
    num_pop = len(pop_0)
    I_0 = np.count_nonzero(pop_0==2)        

    # número de instantes no tempo e passos de tempo 
    num_t = len(tempos)
    passos_de_tempo = tempos[1:] - tempos[:-1]

    # inicializa variáveis para o cálculo da média
    S_mean = np.zeros(num_t)
    I_mean = np.zeros(num_t)
    R_mean = np.zeros(num_t)

    # inicializa variáveis para o cálculo do desvio padrão
    S_sigma = np.zeros(num_t)
    I_sigma = np.zeros(num_t)
    R_sigma = np.zeros(num_t)
    
    # obtém matriz de adjacências e o número médio de vizinhos do grafo
    A_adj = nx.to_numpy_matrix(G)

    # prepara gráfico se necessário
    if show in ('nuvem', 'sd', 'media'):    
        # inicializa figura e define eixo vertical
        plt.figure(figsize=(12,6))
        plt.ylim(0, num_pop)
        plt.xlim(tempos[0], tempos[-1])
    
    if show == 'nuvem':
        # alpha para a nuvem de gráficos das diversas simulaçõe
        alpha = min(0.2, 5/num_sim)    

    # simulações
    for j in range(num_sim):

        # inicializa população de cada simulação
        populacao = np.copy(pop_0)
        S = np.array([num_pop - I_0])
        I = np.array([I_0])
        R = np.array([0])
     
        
        # evolui o dia e armazena as novas contagens
        for dt in passos_de_tempo:

            # calcula probabilidades
            prob_infeccao = 1 - np.exp(-mu*dt)
            prob_recuperacao = 1 - np.exp(-gamma*dt)

            populacao = passo_grafo_estruturado(num_pop, populacao, A_adj,
                                                prob_infeccao, prob_recuperacao)
            S = np.hstack([S, np.count_nonzero(populacao==1)])
            I = np.hstack([I, np.count_nonzero(populacao==2)])
            R = np.hstack([R, np.count_nonzero(populacao==3)])
            
        # adiciona as contagens dessa simulação para o cálculo final da média
        S_mean += S
        I_mean += I
        R_mean += R

        # adiciona as contagens dessa simulação para o cálculo final do desvio padrão
        S_sigma += S ** 2
        I_sigma += I ** 2
        R_sigma += R ** 2

        if show == 'nuvem':
            # exibe os gráficos dos dados de cada simulação
            plt.plot(tempos, S, '-', color='tab:green', alpha=alpha)
            plt.plot(tempos, I, color='tab:red', alpha=alpha)
            plt.plot(tempos, R, '-', color='tab:blue', alpha=alpha)
            plt.plot(tempos, num_pop - S, '-', color='tab:gray', alpha=alpha)

    # divide pelo número de evoluções para obter a média
    S_mean /= num_sim
    I_mean /= num_sim
    R_mean /= num_sim

    # ajusta o calcula do desvio padrão
    S_sigma = ( S_sigma / num_sim - S_mean**2 )**.5
    I_sigma = ( I_sigma / num_sim - I_mean**2 )**.5
    R_sigma = ( R_sigma / num_sim - R_mean**2 )**.5

    # exibe os gráficos das médias
    if show in ('nuvem', 'sd', 'media'):
        plt.plot(tempos, S_mean, '-', color='tab:green', label='suscetíveis')
        plt.plot(tempos, I_mean, '-', color='tab:red', label='infectados')
        plt.plot(tempos, R_mean, '-', color='tab:blue', label='recuperados')
        plt.plot(tempos, num_pop - S_mean, '-', color='tab:gray', label='inf.+ rec.')
        
    if show == 'sd':
        plt.fill_between(tempos, S_mean - S_sigma, S_mean + S_sigma, facecolor='tab:green', alpha = 0.2)
        plt.fill_between(tempos, I_mean - I_sigma, I_mean + I_sigma, facecolor='tab:red', alpha = 0.2)
        plt.fill_between(tempos, R_mean - R_sigma, R_mean + R_sigma, facecolor='tab:blue', alpha = 0.2)
        plt.fill_between(tempos, num_pop - S_mean - S_sigma, num_pop - S_mean + S_sigma, facecolor='tab:gray', alpha = 0.2)

        
    if show == 'nuvem':
        plt.title('Evolução do conjunto de simulações e da média', fontsize=16)
    elif show == 'sd':
        plt.title('Evolução da média, com o desvio padrão', fontsize=16)
    elif show == 'media':
        plt.title('Evolução da média das simulações', fontsize=16)

        
    # informações para o gráfico
    if show in ('nuvem', 'sd', 'media'):
        plt.xlabel('tempo', fontsize=14)
        plt.ylabel('número de indivíduos', fontsize=14)
        plt.legend(fontsize=12)
        plt.show() 

    resultado = SIR_Individual(
        pop_0,
        num_pop,
        mu,
        gamma,
        G,
        tempos,
        num_sim,
        S_mean,
        I_mean, 
        R_mean,
        I_sigma,
        R_sigma,
        S_sigma
    )

    return resultado

In [41]:
a = np.array([1,2,3,4,5])
b = np.array([6,7,8,9,10])

In [94]:
np.select([np.random.rand(len(a)) < 0.5], [a])

array([1, 0, 3, 4, 0])

In [71]:
np.select([np.logical_and(a < 4, b %2 == 0)], [a])

array([1, 0, 3, 0, 0])

In [69]:
np.random.rand(5)

array([0.70252813, 0.41059049, 0.18816847, 0.88813461, 0.79877876])